In [1]:
import Pkg; Pkg.activate(joinpath(@__DIR__,"..")); Pkg.instantiate();
using JLD2
const resfile = joinpath(@__DIR__, "q3.jld2")
const isautograder = @isdefined autograder;

  Activating project at `~/Classes/16745_OptimalControl/hw1_solutions`


# A Falling Brick: Solving Quadratic Programs (40 pts)
In this question we'll be simulating a brick falling and sliding on ice in 2D. You will show that this problem can be formulated as a QP, which you will solve using an Augmented Lagrangian method.

## The Dynamics
The dynamics of the brick can be written in continuous time as
$$ M \dot{v}  + M g = J^T \lambda \\ \text{ where } M = mI, \; g = \begin{bmatrix} 0 \\ 9.81 \end{bmatrix},\; J = \begin{bmatrix} 0 & 1 \end{bmatrix} $$
and $\lambda \in \mathbb{R}$ is the normal force. The velocity $v \in \mathbb{R}^2$ and position $q \in \mathbb{R}^2$ are composed of the horizontal and vertical components.

We can discretize the dynamics with backward Euler:
$$ \begin{align} \begin{bmatrix} v_{k+1} \\ q_{k+1} \end{bmatrix} = \begin{bmatrix} v_k \\ q_k \end{bmatrix}
+ h \begin{bmatrix} \frac{1}{m} J^T \lambda_{k+1} - g \\ v_{k+1} \end{bmatrix} \end{align} $$

We also have the following contact constraints:
$$ \begin{align}
J q_{k+1} &\geq 0 &&\text{(don't fall through the ice)} \\
\lambda_{k+1} &\geq 0 &&\text{(normal forces only push, not pull)} \\
\lambda_{k+1} J q_{k+1} &= 0 &&\text{(no force at a distance)}
\end{align} $$

## Part (a): QP formulation (5 pts)
Show that these discrete-time dynamics are equivalent to the following QP by writing down the KKT conditions.

$$ \begin{align}
    &\text{minimize}_{v_{k+1}} && \frac{1}{2} v_{k+1}^T M v_{k+1} + v_{k+1}^T M (hg - v_k) \\
    &\text{subject to} && -J(q_k + h v_{k+1}) \leq 0 \\
\end{align} $$

**TASK**: Write down the KKT conditions for the optimization problem above, and show that it's equivalent to the dynamics problem stated previously. Use LaTeX markdown. (hint: $q_{k+1} = q_k + hv_{k+1}$)

**SOLUTION**: \
Lagrangian:
$$ \mathcal{L} = \frac{1}{2} v_{k+1}^T M v_{k+1} + v_{k+1}^T M (hg - v_k) - \lambda J(q_k + h v_{k+1}) $$

KKT Conditions:
$$ \begin{align}
    M v_{k+1} + M(hg - v_k) - \lambda h J^T &= 0 \\
    J(q_k + h v_{k+1}) &\ge 0 \\
    \lambda &\ge 0 \\
    \lambda J(q_k + h v_{k+1}) &= 0 \\
\end{align} $$

Solving the first equation for $v_{k+1}$ we get the backward Euler velocity dynamics:
$$ v_{k+1} = v_k + h\left(\frac{1}{m} \lambda J^T - g\right) $$

Substituting $q_k = q_{k+1} - hv_{k+1}$ into the constraint and complimentarity condition we get the other 2 contact constraints:
$$ 
J(q_{k+1}) \geq 0 \\
\lambda J(q_{k+1}) = 0 
$$

## Part (b): Implement an Augmented Lagrangian QP solver (25 pts)
Now that we've shown that we can formulate the falling brick problem as a QP, write an augmented Lagrangian QP solver.

We've provided the following data structure for storing the problem data for a generic QP of the form:
$$ \begin{align}
    &\text{minimize}_{x} && \frac{1}{2} x^T P x + q^T x \\
    &\text{subject to} && A x = b \\
    &&& C x \leq d \\
\end{align} $$

We've also provided a handful of functions that you may find useful when implementing your augmented Lagrangian solver. You're not required to use them.

In [2]:
using Random, LinearAlgebra
"""
    QPData

Holds the data for a Quadratic Program (QP) of the following form:

min 0.5 x'P*x + q'x
st. A*x = b
    C*x ≤ d

# Constructors
    QPData(P,q,A,b,C,d)
    QPData(n::Int,m::Int,p::Int)

The second constructor will initialize all the problem with zeros of the appropriate dimension
"""
struct QPData
    P::Matrix{Float64}
    q::Vector{Float64}
    A::Matrix{Float64}
    b::Vector{Float64}
    C::Matrix{Float64}
    d::Vector{Float64}
end
function QPData(n::Int, m::Int, p::Int)
    QPData(zeros(n,n), zeros(n), zeros(m,n), zeros(m), zeros(p,n), zeros(p))
end
Base.size(qp::QPData) = (length(qp.q), num_eq(qp), num_ineq(qp))
num_eq(qp::QPData) = length(qp.b)
num_ineq(qp::QPData) = length(qp.d)

objective(qp::QPData, x) = 0.5 * x'qp.P*x + qp.q'x
ceq(qp::QPData, x) = qp.A * x - qp.b
cin(qp::QPData, x) = qp.C * x - qp.d

function primal_residual(qp::QPData, x, λ, μ)
    qp.P*x + qp.q + qp.A'λ + qp.C'μ
end

function dual_residual(qp::QPData, x, λ, μ)
    g = ceq(qp, x)
    h = cin(qp, x)
    return [g; max.(0, h)]
end

function complimentarity(qp::QPData, x, λ, μ)
    return [min.(0, μ); μ .* cin(qp, x)]
end

complimentarity (generic function with 1 method)

Implement the following function, which solves the QP specified by a `QPData` object. See the code below for an example of using the `QPData` type and how we expect it to be passed into the method. You're not required to solve a problem with equality constraints (since the brick problem doesn't require it), but we recommend adding in the functionality so you have a fully-functioning QP solver you can use for other problems.

As we saw in class, an augmented Lagrangian solver consists of two loops: an "inner" loop that takes Newtons steps on the unconstrained augmented Lagrangian, and an "outer" loop that updates the penalty parameter and the estimates of the dual variables. We've provided you some starter code below to help you out. If you want to change those other methods (maybe to use a custom Julia type or take in extra input arguments), you're welcome to do so. We'll only call the outer `solve_qp` method from our test scripts.

In [39]:
# TASK: Implement the following method (25 pts)
"""
    solve_qp(qp::QPData, x0, [λ0, μ0]; kwargs...)

Solve the quadratic program (QP) specified by `qp::QPData`, given initial guess `x` for the primal variables, 
and optionally the Lagrange multipliers for the equality `λ` and inequality `μ` constraints.

Returns the optimized solution of primal and dual variables, `xstar,λstar,μstar`.

# Optional Keyword Arguments
* `penalty_initial` initial value of the penalty parameter
* `penalty_scaling` geometric scaling factor for the penalty updates
* `eps_primal` tolerance for primal feasiblity (constraint violation)
* `eps_inner` tolerance for inner Newton solve
* `max_iters` maximum number of outer loop iterations
"""
function solve_qp(qp::QPData, x0, λ0=zeros(num_eq(qp)), μ0=zeros(num_ineq(qp)); 
        penalty_initial=10.0, 
        penalty_scaling=10.0, 
        eps_primal=1e-6,
        eps_inner=1e-6,
        eps_dual=eps_inner,
        max_iters=20
    )
    x = copy(x0)
    λ = copy(λ0)
    μ = copy(μ0)
    
    ρ = penalty_initial
    ϕ = penalty_scaling
    
    # Start outer loop
    for i = 1:max_iters
        # Solve the inner, unconstrained problem
        x = newton_solve(qp, x, λ, μ, ρ, eps_inner=eps_inner)

        # Use the new solution to update the dual variables
        λ, μ = dual_update(qp, x, λ, μ, ρ)
        
        # TODO: update the penalty parameter
        ρ *= ϕ
        
        if norm(dual_residual(qp, x, λ, μ)) < eps_primal && norm(primal_residual(qp, x, λ, μ)) < eps_dual
            # Return the optimized variables
            return x, λ, μ
        end        
    end
    
    @warn "Outer loop max iterations"
    return x, λ, μ 
end

# Optional Methods you may find useful
"""
    newton_solve(qp, x, λ, μ, ρ; kwargs...)

Minimize the augmented Lagranginan given the current values of the dual variables `λ` and `μ` and the penalty parameter `ρ`.
"""
function newton_solve(qp, x, λ, μ, ρ; eps_inner=1e-6)
    for i = 1:10
        # Compute the gradient and the Hessian of the augmented Lagrangian
        r = algrad(qp, x, λ, μ, ρ)
        if norm(r) < eps_inner
            return x
        end
        H = alhess(qp, x, λ, μ, ρ)
        
        # TODO: Compute the Newton step
        #       A line search will help with convergence, but shouldn't be necessary for our problem since we're providing a good guess each time
        dx = -(H\r)
        x .+= dx
    end
    @warn "Inner solve max iterations"
    return x
end

"""
    algrad(qp, x, λ, μ, ρ)

Compute the gradient of the augmented Lagrangian, provided the QP data `qp`, penalty parameter `ρ`,
primal variables `x`, equality Lagrange multipliers `λ` and inequality Lagrange multipliers `μ`
"""
function algrad(qp, x, λ, μ, ρ)
    # TODO: compute the gradient of the augmented Lagrangian
    # HINT: be sure to compute the active constraints!
    grad = zero(x)
    g = ceq(qp, x)
    h = cin(qp, x)
    Iρ = Diagonal(zero(μ))
    for i = 1:length(μ)
        if h[i] > 0 || μ[i] > 0
            Iρ[i,i] = ρ
        end
    end
    grad = qp.P * x + qp.q + ρ * qp.A'g + qp.A'λ + qp.C'*Iρ*h + qp.C'μ
    return grad
end

"""
    alhess(qp, x, λ, μ, ρ)

Compute the Hessian of the augmented Lagrangian, provided the QP data `qp`, penalty parameter `ρ`,
primal variables `x`, equality Lagrange multipliers `λ` and inequality Lagrange multipliers `μ`
"""
function alhess(qp, x, λ, μ, ρ)
    # TODO: compute the Hessian of the augmented Lagrangian
    n = size(x,1)
    Iρ = Diagonal(zero(μ))
    h = cin(qp, x)
    for i = 1:length(μ)
        if h[i] > 0 || μ[i] > 0
            Iρ[i,i] = ρ
        end
    end
    hess = qp.P + ρ*qp.A'qp.A + qp.C'Iρ*qp.C
    return hess
end

"""
    dual_update(qp, x, λ, μ, ρ)

Update the dual variables `λ` and `μ` give the primal variables `x`, QP data `qp` and penalty parameter `ρ`.
"""
function dual_update(qp, x, λ, μ, ρ)
    # TODO: compute the new values for λ and μ
    λnext = λ + ρ * ceq(qp, x)
    μnext = μ + ρ * cin(qp, x)
    
    # Keep the dual variables for the inequality constraints in the nonnegative orthant
    for i = 1:length(μ)
        μnext[i] = max(0, μnext[i])
    end
    return λnext, μnext
end

dual_update

In [34]:
# BRIANS SOLUTION
include("qpal.jl") # <-- see code here
function solve_qp(qp::QPData, x0, λ0=zeros(num_eq(qp)), μ0=zeros(num_ineq(qp)); 
        penalty_initial=10.0, 
        penalty_scaling=10.0, 
        eps_primal=1e-6,
        eps_inner=1e-6,
        max_iters=20
    )
    n,m,p = size(qp)
    x = SVector{n}(x0)
    λ = SVector{m}(λ0)
    μ = SVector{p}(μ0)

    solver = QPAL(qp, penalty_initial, penalty_scaling)
    return solve(solver, x, λ, μ)
end

solve_qp (generic function with 3 methods)

You can use the following code to test your QP solver.

In [41]:
using Test, Random
Random.seed!(2)

# Setting up and solving a random QP
n,m,p = 10,1,15 
qp = QPData(n,m,p)
P = rand(n,n)
qp.P .= P'P   # make it P.S.D
qp.q .= randn(n)
qp.A .= randn(m,n)
qp.b .= randn(m)
qp.C .= randn(p,n)
qp.d .= randn(p)

# Initial guess
x0 = randn(n)

xstar, λstar, μstar = solve_qp(qp, x0)

# # Check optimality conditions
@test norm(primal_residual(qp, xstar, λstar, μstar)) < 1e-3
@test norm(dual_residual(qp, xstar, λstar, μstar)) < 1e-6
@test norm(complimentarity(qp, xstar, λstar, μstar)) < 1e-3;

In [46]:
@testset "3b" begin  # POINTS = 25
    Random.seed!(4)
    # Setting up and solving a random QP
    n,m,p = 10,2,15 
    qp = QPData(n,m,p)
    P = rand(n,n)
    qp.P .= P'P   # make it P.S.D
    qp.q .= randn(n)
    qp.A .= randn(m,n)
    qp.b .= randn(m)
    qp.C .= randn(p,n)
    qp.d .= randn(p)

    # Initial guess
    x = randn(n)

    # Solve
    xstar, λstar, μstar = solve_qp(qp, x)
    
    # Check optimality conditions
    @test norm(primal_residual(qp, xstar, λstar, μstar)) < 1e-3  # POINTS = 5
    @test norm(dual_residual(qp, xstar, λstar, μstar)) < 1e-6    # POINTS = 5
    @test norm(complimentarity(qp, xstar, λstar, μstar)) < 1e-3  # POINTS = 5
    
    # Compare with OSQP
    using OSQP, SparseArrays
    model = OSQP.Model()
    OSQP.setup!(model, P=sparse(qp.P), q=qp.q, A=sparse([qp.A; qp.C]), l=[qp.b; fill(-Inf,p)], u=[qp.b; qp.d],
        eps_abs=1e-6, eps_rel=1e-6, verbose=true)
    res = OSQP.solve!(model)
#     @test norm(res.x - xstar) < 1e-3           # POINTS = 5
#     @test norm(res.y - [λstar; μstar]) < 1e-3  # POINTS = 5
end;

Test Summary: | Pass  Total
3b            |    3      3
-----------------------------------------------------------------
           OSQP v0.6.2  -  Operator Splitting QP Solver
              (c) Bartolomeo Stellato,  Goran Banjac
        University of Oxford  -  Stanford University 2021
-----------------------------------------------------------------
problem:  variables n = 10, constraints m = 17
          nnz(P) + nnz(A) = 225
settings: linear system solver = qdldl,
          eps_abs = 1.0e-06, eps_rel = 1.0e-06,
          eps_prim_inf = 1.0e-04, eps_dual_inf = 1.0e-04,
          rho = 1.00e-01 (adaptive),
          sigma = 1.00e-06, alpha = 1.60, max_iter = 4000
          check_termination: on (interval 25),
          scaling: on, scaled_termination: off
          warm start: on, polish: off, time_limit: off

iter   objective    pri res    dua res    rho        time
   1  -7.2126e+00   4.13e+00   5.61e+02   1.00e-01   1.01e-04s
 200   3.0915e+01   2.57e-04   4.99e-04   9.58e-01   3

## Part (c): Simulate the system (10 pts)
Use your solver from the previous question to simulate the brick for 3 seconds, from the initial condition of `q0 = [0,1]`, `v0 = [1,0]` with `h=0.01` sec and `m=1`.
Use the provided visualization code to visualize your results.

**NOTE**: If you are unable to get your QP solver to work, feel free to use OSQP to solve the QP. An example of setting up and solving a QP with OSQP is provided above.

In [7]:
# TASK: Implement the following method (2 pts)
"""
    build_qp(q, v; mass=1, h=0.01)

Build the Quadratic Program corresponding to the falling brick example of mass `mass`, 
given the 2D position `q` and velocity `v`, and the time step `h`.

Should return a `QPData` object with the correct sizes.
"""
function build_qp(q,v; mass=1, h=0.01)
    # TODO: finish the function
    Pmat = zeros(2,2)
    qvec = zeros(2)
    A = zeros(0,2)
    b = zeros(0)
    C = zeros(1,2)
    d = zeros(1)
    
    # SOLUTION
    M = I(2)*mass
    g = [0,9.81]
    J = [0 1]
    Pmat .= Matrix(M)
    qvec .= M*(h*g - v)
    C .= -J*h
    d .= J*q
    
    # Return as a QPData type
    QPData(Pmat,qvec,A,b,C,d)
end

# SOLUTION: Extra function for using the simulation
function update_qp!(qp, q, v)
    M = qp.P
    h = -qp.C[2]
    g = [0,9.81]
    qp.q .= M*(h*g - v)
    qp.d .= q[2]
    return nothing
end

update_qp! (generic function with 1 method)

In [8]:
@testset "3c" begin                                # POINTS = 10
    @testset "build qp" begin                      # POINTS = 2
        q = [1.2,-0.36]
        v = [10,-1.2]
        qp = build_qp(q, v)
        @test qp.P ≈ load(resfile, "P") atol=1e-6  # POINTS = 0.5
        @test qp.q ≈ load(resfile, "q") atol=1e-6  # POINTS = 0.5
        @test qp.A ≈ load(resfile, "A") atol=1e-6  # POINTS = 0.25
        @test qp.b ≈ load(resfile, "b") atol=1e-6  # POINTS = 0.25
        @test qp.C ≈ load(resfile, "C") atol=1e-6  # POINTS = 0.25
        @test qp.d ≈ load(resfile, "d") atol=1e-6  # POINTS = 0.25
    end
end;

Test Summary: | Pass  Total
3c            |    6      6


In [213]:
# TASK: Implement the following method (8 pts)
function simulate_brick(q0=[0,1.], v0=[1,0.]; h=0.01, T=3.0, m=1.0)
    times = range(0, T, step=h)
    qs = [zero(q0) for t in times]
    vs = [zero(v0) for t in times]
    qs[1] .= q0
    vs[1] .= v0
    λs = zeros(length(times))
    
    # TODO: Simulate the brick by solving the QP
    #  TIP: remember to update your QP after each step
    
    # SOLUTION
    # Build QP
    qp = build_qp(q0, v0; mass=m, h=h)
    g = [0,9.81]

    # Simulation Loop
    for i = 1:length(times)-1
        # Update the qp with the new values
        update_qp!(qp, qs[i], vs[i])
        
        # Solve the QP for the next velocity
        vnext,_,λ = solve_qp(qp, vs[i])
        λs[i] = λ[1]
        vs[i+1] .= vnext
        
        # Use backward Euler to propagate the state
        qs[i+1] .= qs[i] + h*vs[i+1]
    end
    
    # Return the state and velocity trajectories
    return qs, vs, λs
end

simulate_brick (generic function with 3 methods)

### Visualize the Results
Use the following code to visualize the the results of your simulation

In [156]:
# Set up Visualizer
using MeshCat
using GeometryBasics, Colors, CoordinateTransformations
if !isautograder
    vis = Visualizer()
    setobject!(vis["brick"], Rect3D(Vec(0,0,0f0), 0.5*Vec(2,1,1f0)), MeshPhongMaterial(color=colorant"firebrick"))
    render(vis)
end

┌ Info: MeshCat server started. You can open the visualizer by visiting the following URL in your browser:
│ http://127.0.0.1:8701
└ @ MeshCat /home/brian/.julia/packages/MeshCat/Ax8pH/src/visualizer.jl:73


MeshCat.DisplayedVisualizer(MeshCat.CoreVisualizer(MeshCat.SceneTrees.SceneNode(nothing, nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("meshcat" => MeshCat.SceneTrees.SceneNode(nothing, nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("brick" => MeshCat.SceneTrees.SceneNode(UInt8[0x83, 0xa6, 0x6f, 0x62, 0x6a, 0x65, 0x63, 0x74, 0x84, 0xaa  …  0x68, 0x63, 0x61, 0x74, 0x2f, 0x62, 0x72, 0x69, 0x63, 0x6b], nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}()))))), Set{Any}(), ip"127.0.0.1", 8701))

In [157]:
function show_sim(vis, qs, h)
    fps = Int(1/h)
    anim = MeshCat.Animation(fps)
    for (i,q) in enumerate(qs)
        atframe(anim, i) do
            settransform!(vis["brick"], Translation(q[1],0,q[2]))
        end
    end
    setanimation!(vis, anim)
end
if !isautograder
    show_sim(vis, h::Real) = show_sim(vis, simulate_brick(h=h)[1], h)
    show_sim(vis, 0.01);
end

In [158]:
using Statistics
@testset "3c" begin                  # POINTS = 10      
    @testset "simulate brick" begin  # POINTS = 8
        h = 0.01
        qans = load(resfile, "qs")
        vans = load(resfile, "vs")
        qs, vs = simulate_brick(h=h)
        eps = 1e-6

        @test [q[1]/0.01 for q in diff(qs)] ≈ [v[1] for v in vs[1:end-1]] atol=1e-6  # Sanity check velocities              POINTS = 0.5
        @test std([q[1] for q in diff(qs)]) < eps                                    # no horizontal acceleration           POINTS = 0.5
        @test all(q->q[1] > 0, diff(qs))                                             # positive horizontal velocity         POINTS = 0.5
        @test all(q->q[2] > -eps, qs)                                                # no penetration through the floor     POINTS = 1
        @test all(v->v[1] ≈ 1.0, vs)                                                 # constant horizontal velocity         POINTS = 0.5
        @test all(v->v[2] < eps, vs)                                                 # all vertical velocity is negative    POINTS = 1
        @test all(v->abs(v[2]) < eps, vs[101:end])                                   # zero vertical velocity after impact (actual impact time is before this)  # POINTS = 1
        @test qs ≈ qans atol=1e-3  # POINTS = 1.5
        @test vs ≈ vans atol=1e-3  # POINTS = 1.5
    end
end;

Test Summary: | Pass  Total
3c            |    9      9


## EXTRA CREDIT: Make it fast! (max 15 pts)
You can earn extra credit by making your QP solver fast. Points will be given relative to the speed of OSQP, a state-of-the-art QP solver. There will be four different levels:
1. Less than 0.5x the time of OSQP (2x slower that OSQP) (2 pts)
2. Faster than OSQP (5 pts)
3. 2x faster than OSQP (8 pts)
4. Faster than Brian's solution (about 5x faster than OSQP) (10 pts)

It will be timed on the brick simulator. Further extra credit (5 pts) may be assigned if you implement equality constraints and show it's able to successfully solve them.

Tips:
* Check out the `StaticArrays` package
* Consider making your own solver type
* Avoid allocating new memory
* Use the `BenchmarkTools` package to check the performance of individual pieces
* Check out the [Julia Performance Tips](https://docs.julialang.org/en/v1/manual/performance-tips/)
* Write a version of your simulation code that uses OSQP to compare performance

In [13]:
# Sample timing results
using BenchmarkTools
println("Student solution")
@btime simulate_brick();

Student solution
  5.264 ms (46236 allocations: 2.38 MiB)


In [14]:
# SOLUTION
include("qpal.jl") # <-- see code here
function simulate_brick_brian(q0=SA[0,1.], v0=SA[1,0.]; h=0.01, T=3.0, m=1.0)
    times = range(0, T, step=h)
    qs = [zero(q0) for t in times]
    vs = [zero(v0) for t in times]
    qs[1] = q0
    vs[1] = v0
    
    # TODO: Simulate the brick by solving the QP
    #  TIP: remember to update your QP after each step
        
    # Build QP solver
    M = SA[m 0; 0 m]
    g = SA[0,9.81]
    J = SA[0 1.]
    P = Symmetric(M)
    q = M*(h*g - v0)
    A = @SMatrix zeros(Float64,0,2)
    b = @SVector zeros(Float64,0)
    C = -J*h
    d = J*q0
    a = @MVector zeros(Bool,1)
    solver = QPAL(P,q,A,b,C,d,a, 10.0, 10.0)
    
    # Initialize dual variables
    λ = @SVector zeros(Float64,0)
    μ = @SVector zeros(Float64,1)

    # Simulation Loop
    for i = 1:length(times)-1
        # Update the qp with the new values
        solver.q = M*(h*g - vs[i])
        solver.d = SA[qs[i][2]]
        
        # Solve the QP for the next velocity
        solver.ρ = 100.0   # reset the penalty parameter
        vnext, = solve(solver, vs[i], λ, μ)
        vs[i+1] = vnext
        
        # Use backward Euler to propagate the state
        qs[i+1] = qs[i] + h*vs[i+1]
    end
    return qs, vs
end

function simulate_brick_OSQP(q0=[0,1.], v0=[1,0.]; h=0.01, T=3.0, m=1.0)
    times = range(0, T, step=h)
    qs = [zero(q0) for t in times]
    vs = [zero(v0) for t in times]
    qs[1] .= q0
    vs[1] .= v0

    # Build QP
    qp = build_qp(q0, v0; mass=m, h=h)
    n,m,p = size(qp)
    g = [0,9.81]
    model = OSQP.Model()
    OSQP.setup!(model, P=sparse(qp.P), q=qp.q, A=sparse([qp.A; qp.C]), l=[qp.b; fill(-Inf,p)], u=[qp.b; qp.d],
        eps_abs=1e-6, eps_rel=1e-6, verbose=false)

    # Simulation Loop
    for i = 1:length(times)-1
        # Update the qp with the new values
        update_qp!(qp, qs[i], vs[i])
        OSQP.update!(model, q=qp.q, u=qp.d)
        
        # Solve the QP for the next velocity
        res = OSQP.solve!(model)
        vs[i+1] .= res.x
        
        # Use backward Euler to propagate the state
        qs[i+1] .= qs[i] + h*vs[i+1]
    end
    return qs, vs
end

simulate_brick_OSQP (generic function with 3 methods)

In [20]:
@btime simulate_brick_brian();

  50.438 μs (4 allocations: 9.88 KiB)


In [21]:
println("OSQP Solution")
@btime simulate_brick_OSQP();

  851.432 μs (6408 allocations: 445.70 KiB)
